<a href="https://colab.research.google.com/github/yvrjsharma/HugginFace_Gradio/blob/main/PhotoMaker_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/TencentARC/PhotoMaker.git
%cd PhotoMaker
# Install requirements
!pip install -r requirements.txt
# Install photomaker
!pip install git+https://github.com/TencentARC/PhotoMaker.git

from photomaker import PhotoMakerStableDiffusionXLPipeline
from huggingface_hub import hf_hub_download
import torch
import os
from diffusers.utils import load_image
from diffusers import EulerDiscreteScheduler
from photomaker import PhotoMakerStableDiffusionXLPipeline
from PIL import Image
import gradio as gr

photomaker_path = hf_hub_download(repo_id="TencentARC/PhotoMaker", filename="photomaker-v1.bin", repo_type="model")
base_model_path = 'SG161222/RealVisXL_V3.0'
device = "cuda" if torch.cuda.is_available() else "cpu"

### Load base model
pipe = PhotoMakerStableDiffusionXLPipeline.from_pretrained(
    base_model_path,  # can change to any base model based on SDXL
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    variant="fp16"
).to(device)

### Load PhotoMaker checkpoint
pipe.load_photomaker_adapter(
    os.path.dirname(photomaker_path),
    subfolder="",
    weight_name=os.path.basename(photomaker_path),
    trigger_word="img"  # define the trigger word
)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

In [19]:
def generate_image(files, prompt, negative_prompt):
  # load the input ID images
  files_list = []
  for f in files:
    img = Image.open(f.name)
    files_list.append(img)
  # Note that the trigger word `img` must follow the class word for personalization
  generator = torch.Generator(device=device).manual_seed(42)
  images = pipe(
      prompt=prompt,
      input_id_images=files_list,
      negative_prompt=negative_prompt,
      num_images_per_prompt=1,
      num_inference_steps=50,
      start_merge_step=10,
      generator=generator,).images[0]
  return [images]

In [21]:
logo = r"""<center><img src='https://photo-maker.github.io/assets/logo.png' alt='PhotoMaker logo' style="width:80px; margin-bottom:10px"></center>"""
title = r"""<h1 align="center">PhotoMaker: Customizing Realistic Human Photos via Stacked ID Embedding</h1>"""
with gr.Blocks() as demo:
    with gr.Row():
      gr.Markdown(title)
      gr.Markdown(logo)
    with gr.Row():
        files = gr.Files(
                    label="Drag (Select) 1 or more photos of your face",
                    file_types=["image"])
        gallery = gr.Gallery(label="Generated Images")
    with gr.Row():
        prompt = gr.Textbox(label="Prompt",
                  info="Try something like 'a photo of a man/woman img', 'img' is the trigger word.",
                  placeholder="A photo of a [man/woman img]...")
        negative_prompt = gr.Textbox(
                label="Negative Prompt",
                placeholder="low quality",
                value="nsfw, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry",)
        submit = gr.Button("Submit")


    submit.click(generate_image, [files, prompt, negative_prompt], [gallery])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1443074086d29b3276.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
